In [1]:
import copy
import itertools
import os
import re
import shutil
import subprocess
from datetime import datetime
from pathlib import Path

import pandas as pd
import pygrok
from tqdm.notebook import tqdm

## Features
- [x] FileDB: an index for lookup of paths, file name parts and patterns, also for offline use.
- [x] Preprocess using updatedb/plocate RegExp functions (fast)
- [x] Process using python RegExp functions (slow)
- [x] Save as shared file (DB, cache)

In [2]:
def filedb_query_updatedb(
    updatedb_path=[
        "interfaces/filedb/data/updatedb/urbisphere_RAW.db",
        "interfaces/filedb/data/updatedb/urbisphere_L1.db",
    ],
    patterns_chain=[
        ["PARIS", "BERLIN", "FREIBURG", "BRISTOL"],
        ["TIMESTAMP_FILENAME", "TIMESTAMP_DWL_BG", "TIMESTAMP_ALC_CL31"],
    ],
    replace_cache=False,
    verbose=False,
):
    def rebase_path(path_base="urbisphere-dm", path_root=None):
        """return abs path of a higher level directory"""
        from pathlib import Path

        path_root = Path("__file__").parent.resolve() if not path_root else path_root
        path_parts = lambda p: p[
            0 : (p.index(path_base) + 1 if path_base in p else len(p))
        ]
        return str(Path(*[n for n in path_parts(Path(path_root).parts)]))

    def convert_search_pattern(
        py_regex_pattern, predefined_patterns, regex_type="plocate"
    ):
        """lookup search patterns; modified from pygrok"""
        type_mapper = {}

        if regex_type == "python":
            re_sub_fun = (
                lambda m: "(?P<"
                + m.group(2)
                + ">"
                + predefined_patterns[m.group(1)].regex_str
                + ")"
            )
        else:
            re_sub_fun = lambda m: "(" + predefined_patterns[m.group(1)].regex_str + ")"

        while True:
            # replace %{pattern_name:custom_name} (or %{pattern_name:custom_name:type}
            # with regex and regex group name

            py_regex_pattern = re.sub(
                r"%{(\w+):(\w+)(?::\w+)?}",
                re_sub_fun,
                py_regex_pattern,
            )

            # replace %{pattern_name} with regex
            py_regex_pattern = re.sub(
                r"%{(\w+)}",
                lambda m: "(" + predefined_patterns[m.group(1)].regex_str + ")",
                py_regex_pattern,
            )

            if re.search(r"%{\w+(:\w+)?}", py_regex_pattern) is None:
                break

        regex_obj = re.compile(py_regex_pattern)

        return regex_obj

    # Convert custom patterns
    def custom_regex(custom_patterns):
        """break down pygrok custom patterns into regex strings for plocate and python, as comprehensive as possible"""
        custom_pats = {}

        for pat_name, regex_str in custom_patterns.items():
            custom_pats[pat_name] = pygrok.Pattern(pat_name, regex_str)

        # Translation of custom and default patterns to python and normal flavour REGEX
        Grok = pygrok.Grok(
            "example",
            custom_patterns_dir="../conf/pygrok/",
            custom_patterns=custom_patterns,
        )
        predefined_patterns = Grok.predefined_patterns

        custom_patterns_plocate = {
            k: convert_search_pattern(
                predefined_patterns[k].regex_str,
                predefined_patterns,
                regex_type="plocate",
            )
            for k, v in custom_patterns.items()
        }
        custom_patterns_py = {
            k: convert_search_pattern(
                predefined_patterns[k].regex_str,
                predefined_patterns,
                regex_type="python",
            )
            for k, v in custom_patterns.items()
        }

        custom_patterns = {k: v.pattern for k, v in custom_patterns_plocate.items()}

        return (custom_patterns, custom_patterns_plocate, custom_patterns_py)

    def plocate_cmd(
        pattern,
        predefined_patterns,
        updatedb_path="urbisphere_RAW.db",
        count_only=False,
        basename_only=False,
    ):
        """Return plocate shell commands"""
        if isinstance(pattern, str):
            pattern = [pattern]

        import copy
        import shutil

        plocate_cmd = [shutil.which(c) for c in ["plocate","locate"] if shutil.which(c)][0]
        

        run_cmd = [
            str(plocate_cmd),
            "-d",
            "'{}'".format(
                ":".join(updatedb_path)
                if isinstance(updatedb_path, list)
                else str(updatedb_path)
            ),
            "--null",
        ]

        if basename_only:
            run_cmd.append("-b")

        if count_only:
            run_cmd.append("-c")

        run_p = []
        run_cmds = []
        for p in pattern:
            if p in predefined_patterns:
                pattern_regex = (
                    predefined_patterns[p].pattern
                    if isinstance(predefined_patterns[p], re.Pattern)
                    else predefined_patterns[p]
                )
            else:
                return []

            run_cmd.append("--regex")
            run_cmd.append("'{}'".format(pattern_regex))
            run_cmds.append(copy.deepcopy(run_cmd))
            run_p.append(p)

        run_dict = dict(zip(run_p, run_cmds))
        return run_dict

    def match_dict(text, regex_obj):
        """If text is matched with pattern, return variable names specified(%{pattern:variable name})
        in pattern and their corresponding values.If not matched, return None.
        """

        match_obj = regex_obj.search(text)

        if match_obj == None:
            return {}
        matches = match_obj.groupdict()
        return matches

    # filters - years
    def func_year(x):
        if ~isinstance(x, float):
            if isinstance(x, str):
                if x.startswith("Q"):
                    # Vaisala years
                    x = "202" + x[1:]
                elif len(x) == 2:
                    # Decadal years
                    x = "20" + x
        return x

    # filters - datetime
    def func_date(row):
        date_cols = ["time_start_year", "time_start_month", "time_start_day"]
        time_cols = ["time_start_hour", "time_start_minute", "time_start_second"]
        if not any(
            [isinstance(row[k], float) for k in date_cols + time_cols]
        ) and not any(
            [pd.isna(row[k]) for k in time_cols]
        ):  # date and time
            return datetime(*[int(row[k]) for k in date_cols + time_cols])
        elif not any([isinstance(row[k], float) for k in date_cols]) and not all(
            [pd.isna(row[k]) for k in time_cols]
        ):  # date and partial time
            return datetime(
                *[
                    int(row[k]) if not isinstance(row[k], float) else int(0)
                    for k in date_cols + time_cols
                ]
            )
        elif not any([isinstance(row[k], float) for k in date_cols]) and not any(
            [pd.isna(row[k]) for k in date_cols]
        ):  # date only
            return datetime(*[int(row[k]) for k in date_cols])
        else:
            return None

    def filter_dataframe(df):
        # deduplicate columns
        for k in tqdm(
            list(dict.fromkeys(df.columns[df.columns.duplicated(keep=False)]))
        ):
            print(f"Filter {k}")
            ds = df.loc[:, k]
            ds = ds.stack().groupby(level=0).first().reindex(ds.index)
            df = df.drop(columns=k)
            df[k] = ds

        # apply filters
        for k in ["time_start_year"]:
            print(f"Filter {k}")
            df[k] = df[k].apply(lambda x: func_year(x))

        for k in ["time"]:
            print(f"Filter {k}")
            df[k] = df.apply(lambda row: func_date(row), axis=1)

        return df

    def cache_file_name(updatedb_path):
        L = list(itertools.chain(*[Path(n).stem.split("_") for n in updatedb_path]))
        L = list(dict.fromkeys(L))
        P = list(itertools.chain(*[[str(Path(n).parent)] for n in updatedb_path]))
        P = list(dict.fromkeys(P))[0]
        E = list(itertools.chain(*[[str(Path(n).suffix)] for n in updatedb_path]))
        E = list(dict.fromkeys(E))[0]

        return Path(os.path.join(P, "_".join(L) + E)).with_suffix(".feather")

    def export_feather(df, cache_path):
        df.columns = [str(x) for x in df.columns.to_flat_index()]
        df = df.reset_index()
        df.to_feather(cache_path)

    def import_feather(cache_path):
        from ast import literal_eval as make_tuple

        df = pd.read_feather(cache_path)
        df = df.set_index("file_path")
        df.columns = pd.MultiIndex.from_tuples([make_tuple(n) for n in df.columns])
        return df

    # plocate accepts ERE, but without atomic grouping
    custom_patterns = {
        # file system
        "UNIXPATH": r"(/([\w_%!$@:.,~-]+|\\.)*)+",
        "UNIXPATH2": "(([^/]+[/])+)",
        "UNIXFILE": ".*?[a-zA-Z0-9]$",
        "UNIXFILE2": r"urbishere_[\w_%!$@:.,~-]*$",        
        "NULL": "\x00",
        # file locations
        "REPOSITORY": "^(/srv/meteo/archive/urbisphere/data/)",
        "PRODUCTION": "((RAW|L0|L1|L2)/)",
        # networks
        "SMUROBS": "(by-source/smurobs/)",
        "UKMO": "(by-source/ukmo/)",
        "DWD": "(by-source/dwd/)",
        "SOURCES": "(%{SMUROBS}|%{UKMO}|%{DWD})",
        # campaigns
        "PA": "((by-location|by-serialnr)/France/Paris/)",
        "BE": "((by-location|by-serialnr)/Germany/Berlin/)",
        "FR": "((by-location|by-serialnr)/Germany/Freiburg/)",
        "BR": "((by-location|by-serialnr)/UK/Bristol/)",         
        # sensors, systems
        "SUBPATH": "(%{UNIXPATH2})",
        # time blocks
        "YEAR": "([2][0][2][1-7])",  # only 2021 to 2027!
        "YEAR2": "([0-9][0-9])",
        "YEAR2VAISALA": "([Q][1-5]){1}",  # A calendar in Helsinki; only 2021 to 2025!
        "YEAR4": "([1-2][0-9][0-9][0-9])",
        "MONTHNUM": "(0?[1-9]|1[0-2])",
        "MONTHNUM2": "(0[1-9]|1[0-2])",
        "MONTHDAY2": "((0[1-9])|([12][0-9])|(3[01]))",
        "EPOCHSECONDS": "([0-9]{10}){1}",
        "HOUR": "(2[0123]|[01]?[0-9])",
        "MINUTE": "([0-5][0-9])",
        "SECOND": "(([0-5]?[0-9]|60)([:.,][0-9]+)?)",  # '60' is a leap second in most time standards and thus is valid.
        # compounded time blocks
        "TIME": "(?!<[0-9])%{HOUR}:%{MINUTE}(:%{SECOND})(?![0-9])",
        "ISO8601_TIMEZONE": "(Z|[+-]%{HOUR}(:?%{MINUTE}))",
        "ISO8601_TIMESTAMP": "%{YEAR}-%{MONTHNUM}-%{MONTHDAY}[T ]%{HOUR}:?%{MINUTE}(:?%{SECOND})?%{ISO8601_TIMEZONE}?",
        "ISO8601_TIMESTAMP_CONDENSED": "%{YEAR}%{MONTHNUM2}%{MONTHDAY2}[T]%{HOUR}%{MINUTE}%{SECOND}%{ISO8601_TIMEZONE}?",
        "ISO8601_TIMEBOUND_CONDENSED": "%{ISO8601_TIMESTAMP_CONDENSED}_%{ISO8601_TIMESTAMP_CONDENSED}",
        "TIMESTAMP_FILENAME": r"%{YEAR:time_start_year:int}[-]?%{MONTHNUM2:time_start_month:int}[-]?%{MONTHDAY2:time_start_day:int}[_-]?(%{HOUR:time_start_hour:int}[-]?%{MINUTE:time_start_minute:int}[-]?%{SECOND:time_start_second:int}(%{ISO8601_TIMEZONE:time_start_tzinfo:str})?)?[\.]?",
        "TIMESTAMP_DWL_BG": r"(Background)[_]%{MONTHDAY2:time_start_day:int}%{MONTHNUM2:time_start_month:int}%{YEAR2:time_start_year:int}[-]%{HOUR:time_start_hour:int}%{MINUTE:time_start_minute:int}%{SECOND:time_start_second:int}[\.]",  #'Background_080124-210017'
        "TIMESTAMP_ALC_CL31": r"[_]%{YEAR2VAISALA:time_start_year:int}%{MONTHNUM2:time_start_month:int}%{MONTHDAY2:time_start_day:int}%{HOUR:time_start_hour:int}[\.]",  #'CL31/S2010717/S2010717_Q3113006.dat''
        # compounded other
        "PARIS": "%{REPOSITORY:repository_path:str}%{PRODUCTION:production_path:str}%{SOURCES:source_path:str}%{PA:campaign_path:str}%{SUBPATH:system_path:str}",
        "BERLIN": "%{REPOSITORY:repository_path:str}%{PRODUCTION:production_path:str}%{SOURCES:source_path:str}%{BE:campaign_path:str}%{SUBPATH:system_path:str}",
        "FREIBURG": "%{REPOSITORY:repository_path:str}%{PRODUCTION:production_path:str}%{SMUROBS:source_path:str}%{FR:campaign_path:str}%{SUBPATH:system_path:str}",
        "BRISTOL": "%{REPOSITORY:repository_path:str}%{PRODUCTION:production_path:str}%{SMUROBS:source_path:str}%{BR:campaign_path:str}%{SUBPATH:system_path:str}",                
    }

    # absolute paths
    updatedb_path = [os.path.join(rebase_path(), n) for n in updatedb_path]

    # cache path
    cache_path = cache_file_name(updatedb_path)
    print(cache_path)

    # cache
    if cache_path.exists() and not replace_cache:
        # if a buffered copy exists, read it.
        # print("Reading cache...")
        df = import_feather(cache_path)
        return df

    # helper config
    patterns_sets = list(itertools.product(*patterns_chain))
    updatedb_cwd = Path(updatedb_path[0]).absolute().resolve().parent
    custom_patterns, custom_patterns_plocate, custom_patterns_py = custom_regex(
        custom_patterns
    )

    # plocate commands
    commands_sets = []
    for patterns in patterns_sets:
        cmds = plocate_cmd(
            list(patterns),
            custom_patterns_plocate,
            updatedb_path=":".join([Path(p).name for p in updatedb_path]),
        )
        commands_sets.append(copy.deepcopy(cmds))

    # plocate output
    outputs_sets = []

    for cmds in commands_sets:
        outputs = []
        for k, command in cmds.items():
            command_str = r" ".join(s for s in command)
            if verbose:
                print(f"Command:\n{command_str}")

            with subprocess.Popen(
                [command_str],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                shell=True,
                cwd=updatedb_cwd,
            ) as process:
                output = process.communicate()[0].decode("utf-8")
                returncode = process.returncode
                outputs.append(
                    {"cmd": command_str, "returncode": returncode, "output": output}
                )

        outputs_sets.append(outputs)

    # python regex / pygrok decoding
    pattern_match = {}
    for patterns, outputs in zip(patterns_sets, outputs_sets):
        for n, p in enumerate(patterns):
            new_pattern = patterns[0 : (n + 1)]

            if new_pattern in pattern_match:
                continue

            regex_obj = custom_patterns_py[p]
            output_list = outputs[n]["output"].rstrip("\x00").split("\x00")
            output_list_search = list(filter(regex_obj.search, output_list))
            output_list_result_1 = [
                match_dict(text, regex_obj) for text in output_list_search
            ]
            output_list_result_2 = [
                match_dict(Path(text).name, regex_obj) for text in output_list_search
            ]

            pattern_match[new_pattern] = {
                "text": [],  # output_list
                "search": output_list_search,
                "match_fullpath": output_list_result_1,
                "match_basename": output_list_result_2,
            }

            if verbose:
                print("pattern {}".format(str(new_pattern)))
                print(
                    "text {text} / search {search} / match {match_fullpath} / match basename {match_basename}".format(
                        **{k: len(v) for k, v in pattern_match[new_pattern].items()}
                    )
                )

    # Construct list of dataframes, concatenate list along index
    # (1) Full path
    dx_full = [
        pd.DataFrame(v["match_fullpath"], index=v["search"])
        for k, v in pattern_match.items()
    ]
    df_full = pd.concat(dx_full, axis=1)
    df_full["file_name"] = df_full.index.to_series().apply(lambda x: Path(x).name)
    df_full.index.set_names(["file_path"], inplace=True)

    # (2) Basename only
    dx_base = [
        pd.DataFrame(v["match_basename"], index=v["search"])
        if any(v["match_basename"])
        else None
        for k, v in pattern_match.items()
    ]
    df_base = pd.concat(dx_base, axis=1)
    df_base["file_name"] = df_base.index.to_series().apply(lambda x: Path(x).name)
    df_base.index.set_names(["file_path"], inplace=True)

    # Filter and merge
    df_full = filter_dataframe(df_full)
    df_base = filter_dataframe(df_base)

    df_full.columns = pd.MultiIndex.from_product([["fullpath"]] + [df_full.columns])
    df_base.columns = pd.MultiIndex.from_product([["basename"]] + [df_base.columns])

    df = pd.concat([df_full, df_base], axis=1)

    if not cache_path.exists() or replace_cache:
        # write cache copy
        print("Writing cache...")
        export_feather(df, cache_path)

    return df

In [3]:
if __name__ == "__main__":
    df = filedb_query_updatedb(
        verbose=True,
        replace_cache=True,
        updatedb_path=[
            "interfaces/filedb/data/updatedb/urbisphere_RAW.db",
            "interfaces/filedb/data/updatedb/urbisphere_L0.db",            
            "interfaces/filedb/data/updatedb/urbisphere_L1.db",
            "interfaces/filedb/data/updatedb/urbisphere_L2.db",            
        ],
    )

/home/investigator8/jupyter/urbisphere-dm/interfaces/filedb/data/updatedb/urbisphere_RAW_L0_L1_L2.feather
Command:
/bin/plocate -d 'urbisphere_RAW.db:urbisphere_L0.db:urbisphere_L1.db:urbisphere_L2.db' --null --regex '(^(/srv/meteo/archive/urbisphere/data/))(((RAW|L0|L1|L2)/))((((by-source/smurobs/))|((by-source/ukmo/))|((by-source/dwd/))))(((by-location|by-serialnr)/France/Paris/))((((([^/]+[/])+))))'
Command:
/bin/plocate -d 'urbisphere_RAW.db:urbisphere_L0.db:urbisphere_L1.db:urbisphere_L2.db' --null --regex '(^(/srv/meteo/archive/urbisphere/data/))(((RAW|L0|L1|L2)/))((((by-source/smurobs/))|((by-source/ukmo/))|((by-source/dwd/))))(((by-location|by-serialnr)/France/Paris/))((((([^/]+[/])+))))' --regex '(([2][0][2][1-7]))[-]?((0[1-9]|1[0-2]))[-]?(((0[1-9])|([12][0-9])|(3[01])))[_-]?(((2[0123]|[01]?[0-9]))[-]?(([0-5][0-9]))[-]?((([0-5]?[0-9]|60)([:.,][0-9]+)?))(((Z|[+-]((2[0123]|[01]?[0-9]))(:?(([0-5][0-9]))))))?)?[\.]?'
Command:
/bin/plocate -d 'urbisphere_RAW.db:urbisphere_L0.db:urb

  0%|          | 0/12 [00:00<?, ?it/s]

Filter repository_path
Filter production_path
Filter source_path
Filter campaign_path
Filter system_path
Filter time_start_year
Filter time_start_month
Filter time_start_day
Filter time_start_hour
Filter time_start_minute
Filter time_start_second
Filter time_start_tzinfo
Filter time_start_year


/tmp/ipykernel_3101014/1626567219.py:169: DeprecationWarning: Bitwise inversion '~' on bool is deprecated and will be removed in Python 3.16. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  if ~isinstance(x, float):


Filter time


  0%|          | 0/7 [00:00<?, ?it/s]

Filter time_start_year
Filter time_start_month
Filter time_start_day
Filter time_start_hour
Filter time_start_minute
Filter time_start_second
Filter time_start_tzinfo
Filter time_start_year


/tmp/ipykernel_3101014/1626567219.py:169: DeprecationWarning: Bitwise inversion '~' on bool is deprecated and will be removed in Python 3.16. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  if ~isinstance(x, float):


Filter time
Writing cache...


# DEV

In [4]:
def fnmatch_devel():
    import fnmatch
    import re

    regex = fnmatch.translate("**/*")
    regex

    logger_path = (
        df["logger_path"][~df["logger_path"].str.contains("/by-upload-date/")]
        .unique()
        .tolist()
    )

    df[df[("fullpath", "system_path")].str.contains("/by-upload-date/")].tail(1)

In [5]:
def exceptions(pattern_match):
    # exceptions
    exceptions = (
        set(pattern_match[("PARIS",)]["text"])
        - set(pattern_match[("PARIS", "TIMESTAMP_FILENAME")]["text"])
        - set(pattern_match[("PARIS", "TIMESTAMP_DWL_BG")]["text"])
        - set(pattern_match[("PARIS", "TIMESTAMP_ALC_CL31")]["text"])
    )

    exceptions = [
        f for f in sorted(exceptions) if not Path(f).is_dir() and not "/dupes/" in f
    ]

    len(exceptions)